In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
# Setze den Seed für reproduzierbare Ergebnisse
torch.manual_seed(42)

# Definiere die Transformationen für die Bilder
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Lade den Trainings- und Testdatensatz herunter
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Erstelle DataLoader für das Training und Testen in Batches
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Erste Convolutional Layer: 1 Input-Kanal (Graustufen), 10 Output-Kanäle, Kernel-Größe 5
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0)
        # Zweite Convolutional Layer: 10 Input-Kanäle, 20 Output-Kanäle, Kernel-Größe 5
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0)
        # Dropout-Layer zur Regularisierung
        self.conv2_drop = nn.Dropout2d()
        # Erste Fully Connected Layer
        self.fc1 = nn.Linear(in_features=320, out_features=50)
        # Zweite Fully Connected Layer (Output Layer)
        self.fc2 = nn.Linear(in_features=50, out_features=10)

    # Die forward-Methode definiert, wie die Daten durch das Netzwerk fließen
    # Ein MNIST-Bild startet mit der Form [1, 28, 28] (Kanäle, Höhe, Breite)
    def forward(self, x):
        # Forward-Pass durch die erste Conv-Schicht
        # Input: [1, 28, 28]
        # Output-Größe: (28 - 5) + 1 = 24. Ergebnis-Form: [10, 24, 24]
        x = self.conv1(x)
        
        # Max-Pooling reduziert die Größe
        # Input: [10, 24, 24]
        # Output-Größe: 24 / 2 = 12. Ergebnis-Form: [10, 12, 12]
        x = F.max_pool2d(x, kernel_size=2)

        # ReLU-Aktivierungsfunktion (ändert die Form nicht)
        x = F.relu(x)
        
        # Forward-Pass durch die zweite Conv-Schicht
        # Input: [10, 12, 12]
        # Output-Größe: (12 - 5) + 1 = 8. Ergebnis-Form: [20, 8, 8]
        x = self.conv2(x)

        # Dropout (ändert die Form nicht)
        x = self.conv2_drop(x)

        # Erneutes Max-Pooling
        # Input: [20, 8, 8]
        # Output-Größe: 8 / 2 = 4. Ergebnis-Form: [20, 4, 4]
        x = F.max_pool2d(x, kernel_size=2)

        # ReLU-Aktivierungsfunktion (ändert die Form nicht)
        x = F.relu(x)
        
        # "Flatten" des Tensors für die Fully Connected Layers
        # Input: [20, 4, 4]
        # Die Dimensionen werden zu einem Vektor multipliziert: 20 * 4 * 4 = 320
        # Ergebnis-Form: [320]
        x = x.view(-1, 320)
        
        # Forward-Pass durch die erste FC-Schicht mit ReLU
        # Input: [320]
        # Output: [50]
        x = self.fc1(x)
        x = F.relu(x)
        
        # Forward-Pass durch die Output-Schicht
        # Input: [50]
        # Output: [10] (einer für jede Ziffer von 0-9)
        x = self.fc2(x)
        return x

In [4]:
# Wähle das Gerät (CPU oder GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialisiere das Modell und verschiebe es auf das gewählte Gerät
model = CNN().to(device)

# Definiere den Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definiere die Verlustfunktion
loss_function = nn.CrossEntropyLoss()

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = loss_function(output, target)
        
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Starte das Training für eine bestimmte Anzahl an Epochen
epochs = 5
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.346808
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.540909
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.216896
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.346944
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.192341
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.373181
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.318280
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.330540
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.097440
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.156678
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.223379
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.239331
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.124010
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.084847
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.043524
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.196237
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.137510
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.059350
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.037053
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.138410
Train Epoch:

In [6]:
# Definiert die Funktion zur Evaluierung des Modells
def evaluate(model, device, test_loader):
    # Schaltet das Modell in den Evaluationsmodus. Wichtig, um Layer wie Dropout oder BatchNorm korrekt zu behandeln.
    model.eval()
    
    # Initialisiert Variablen, um den Gesamtverlust und die Anzahl korrekter Vorhersagen zu speichern
    test_loss = 0
    correct = 0
    
    # 'torch.no_grad()' deaktiviert die Gradientenberechnung, was die Auswertung beschleunigt und Speicher spart.
    with torch.no_grad():
        # Schleife über alle Daten-Batches im Test-DataLoader
        for data, target in test_loader:
            # Verschiebt die Daten und die Ziel-Labels auf das aktive Gerät (CPU oder GPU)
            data, target = data.to(device), target.to(device)
            
            # Führt einen Forward-Pass durch: Das Modell macht eine Vorhersage für die Eingabedaten
            output = model(data)
            
            # Berechnet den Verlust für diesen Batch und addiert ihn zum Gesamtverlust. .item() extrahiert den reinen Zahlenwert.
            test_loss += loss_function(output, target).item()
            
            # Findet die Vorhersage des Modells: den Index (die Klasse) mit dem höchsten Wert in der Ausgabe
            pred = output.argmax(dim=1, keepdim=True)
            
            # Vergleicht die Vorhersagen (pred) mit den wahren Labels (target) und zählt, wie viele übereinstimmen
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Berechnet den durchschnittlichen Verlust über den gesamten Testdatensatz
    test_loss /= len(test_loader.dataset)
    
    # Berechnet die Genauigkeit in Prozent
    accuracy = 100. * correct / len(test_loader.dataset)
    
    # Gibt die Ergebnisse formatiert aus
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')

# Führe die Evaluation durch, indem die Funktion mit dem trainierten Modell, dem Gerät und dem Testdatensatz aufgerufen wird
evaluate(model, device, test_loader)


Test set: Average loss: 0.0000, Accuracy: 9879/10000 (98.79%)

